*# Imad Jaddi
*# MISC 1
*# Exercice
*# Système Bancaire


# **---------Exercice : Système Bancaire --------**





 Une **classe** est un modèle (plan) qui permet de créer des objets.
Elle définit :

- des **attributs** (Ce sont les données / variables de la classe)

      --> self.nom
      --> self.age
      --> self.solde

- Un **constructeur** (Méthode spéciale pour initialiser les attributs)

      -->  def __init__(self, nom, age):
      -->      self.nom = nom
      -->      self.age = age



- des **méthodes** (Ce sont les fonctions de la classe)
Elles définissent le comportement de l’objet

      -->  def afficher(self):
      -->     pass


In [ ]:
┌───────────────────────┐
│        Classe         │   ← Nom de la classe
├───────────────────────┤
│        Attributs      │   ← Variables
├───────────────────────┤
│        Méthodes       │   ← Fonctions
└───────────────────────┘


 **Class Abstraction** : L’abstraction permet de définir les fonctionnalités essentielles d’une classe sans détailler leur implémentation. (Cacher les détails)

In [ ]:
from abc import ABC, abstractmethod
from datetime import datetime

# =========================
# Classe abstraite Compte
# =========================
class Compte(ABC):
    def __init__(self, titulaire, solde_initial=0):
        self.titulaire = titulaire
        self.__solde = solde_initial  # attribut privé

    def consulter_solde(self):
        """Retourne le solde actuel"""
        return self.__solde

    def deposer(self, montant, libelle="Dépôt"):
        """Déposer un montant avec libellé optionnel"""
        if montant <= 0:
            raise ValueError("Le montant doit être positif")
        self.__solde += montant
        print(f"{libelle} de {montant} € effectué")

    @abstractmethod
    def retirer(self, montant):
        """Méthode abstraite, doit être définie par les sous-classes"""
        pass

    # Méthode protégée pour modifier le solde
    def _modifier_solde(self, montant):
        self.__solde += montant


# =========================
# Classe Transaction
# =========================
class Transaction:
    def __init__(self, type_operation, montant, libelle=None):
        self.type_operation = type_operation
        self.montant = montant
        self.libelle = libelle
        self.date = datetime.now()

    def __str__(self):
        return f"{self.date} | {self.type_operation} | {self.montant} € | {self.libelle}"


# =========================
# Compte Courant
# =========================
class CompteCourant(Compte):
    def __init__(self, titulaire, solde=0):
        super().__init__(titulaire, solde)
        self.historique = []  # Composition : contient les transactions

    def deposer(self, montant, libelle="Dépôt"):
        super().deposer(montant, libelle)
        self.historique.append(Transaction("Dépôt", montant, libelle))

    def retirer(self, montant):
        if montant <= 0:
            raise ValueError("Montant invalide")
        self._modifier_solde(-montant)
        self.historique.append(Transaction("Retrait", montant))

    def virement(self, destinataire, montant):
        """Effectue un virement vers un autre compte"""
        self.retirer(montant)
        destinataire.deposer(montant, "Virement reçu")
        self.historique.append(Transaction("Virement", montant, f"Vers {destinataire.titulaire.nom}"))


# =========================
# Compte Epargne
# =========================
class CompteEpargne(Compte):
    def __init__(self, titulaire, solde=0, taux_interet=0):
        super().__init__(titulaire, solde)
        self.taux_interet = taux_interet

    def retirer(self, montant):
        if montant > self.consulter_solde():
            raise ValueError("Solde insuffisant")
        self._modifier_solde(-montant)

    def calculer_interet(self):
        """Calcul et ajout des intérêts au solde"""
        interet = self.consulter_solde() * (self.taux_interet / 100)
        self._modifier_solde(interet)
        print(f"Intérêts de {interet:.2f} € ajoutés.")


# =========================
# Classe Client
# =========================
class Client:
    def __init__(self, nom):
        self.nom = nom
        self.comptes = []  # Agrégation

    def ajouter_compte(self, compte):
        self.comptes.append(compte)

    def lister_comptes(self):
        print(f"Comptes de {self.nom} :")
        for compte in self.comptes:
            print(f"{type(compte).__name__} - Solde : {compte.consulter_solde():.2f} €")


In [ ]:
# Création des clients
client1 = Client("Ali")
client2 = Client("Sara")

# Création de comptes
cc = CompteCourant(client1, 1000)
ce = CompteEpargne(client2, 2000, 5)

# Ajout des comptes aux clients
client1.ajouter_compte(cc)
client2.ajouter_compte(ce)

# Opérations
cc.deposer(500, "Salaire")
cc.virement(ce, 300)
ce.calculer_interet()

# Affichage
client1.lister_comptes()
client2.lister_comptes()

# Historique des transactions du compte courant
print("\nHistorique du Compte Courant:")
for t in cc.historique:
    print(t)


Salaire de 500 € effectué
Virement reçu de 300 € effectué
Intérêts de 115.00 € ajoutés.
Comptes de Ali :
CompteCourant - Solde : 1200.00 €
Comptes de Sara :
CompteEpargne - Solde : 2415.00 €

Historique du Compte Courant:
2026-01-16 14:46:51.990716 | Dépôt | 500 € | Salaire
2026-01-16 14:46:51.990781 | Retrait | 300 € | None
2026-01-16 14:46:51.990797 | Virement | 300 € | Vers Sara
